In [1]:
import sys
import os

# Current notebook folder: win_tracker/soccer
notebook_dir = os.getcwd()

# Project root for imports: win_tracker/
project_root = os.path.abspath(os.path.join(notebook_dir, ".."))

# Add to sys.path if not already there
if project_root not in sys.path:
    sys.path.append(project_root)

print("Project root added to sys.path:", project_root)


Project root added to sys.path: /Users/Jake/Documents/GitHub/all-sports-win-tracker/win_tracker


In [2]:
from pyspark.sql import SparkSession
import nflreadpy as nfl
import pandas as pd
import polars as pl

from pyspark.sql.functions import col, lit, when, count, isnan
from spark_setup import get_spark
spark = get_spark()

from func import write_single_csv


Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/11/23 13:42:52 WARN Utils: Your hostname, MacBook-Air-197.local, resolves to a loopback address: 127.0.0.1; using 192.168.1.97 instead (on interface en0)
25/11/23 13:42:52 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


25/11/23 13:42:52 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


25/11/23 13:42:52 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/11/23 13:42:52 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [3]:


# Load current season play-by-play data
pbp = nfl.load_pbp()

# Load player game-level stats for multiple seasons
player_stats = nfl.load_player_stats([2022, 2023])

# Load all available team level stats
team_stats = nfl.load_team_stats(seasons=True)

# nflreadpy uses Polars instead of pandas. Convert to pandas if needed:
pbp_pandas = pbp.to_pandas()



In [4]:
nfl_data = nfl.load_schedules(seasons=True)
display(nfl_data)

game_id,season,game_type,week,gameday,weekday,gametime,away_team,away_score,home_team,home_score,location,result,total,overtime,old_game_id,gsis,nfl_detail_id,pfr,pff,espn,ftn,away_rest,home_rest,away_moneyline,home_moneyline,spread_line,away_spread_odds,home_spread_odds,total_line,under_odds,over_odds,div_game,roof,surface,temp,wind,away_qb_id,home_qb_id,away_qb_name,home_qb_name,away_coach,home_coach,referee,stadium_id,stadium
str,i32,str,i32,str,str,str,str,i32,str,i32,str,i32,i32,i32,str,i32,str,str,i32,str,i32,i32,i32,i32,i32,f64,i32,i32,f64,i32,i32,i32,str,str,i32,i32,str,str,str,str,str,str,str,str,str
"""1999_01_MIN_ATL""",1999,"""REG""",1,"""1999-09-12""","""Sunday""",null,"""MIN""",17,"""ATL""",14,"""Home""",-3,31,0,"""1999091210""",598,null,"""199909120atl""",null,"""190912001""",null,7,7,null,null,-4.0,null,null,49.0,null,null,0,"""dome""","""astroturf""",null,null,"""00-0003761""","""00-0002876""","""Randall Cunningham""","""Chris Chandler""","""Dennis Green""","""Dan Reeves""","""Gerry Austin""","""ATL00""","""Georgia Dome"""
"""1999_01_KC_CHI""",1999,"""REG""",1,"""1999-09-12""","""Sunday""",null,"""KC""",17,"""CHI""",20,"""Home""",3,37,0,"""1999091206""",597,null,"""199909120chi""",null,"""190912003""",null,7,7,null,null,-3.0,null,null,38.0,null,null,0,"""outdoors""","""grass""",80,12,"""00-0006300""","""00-0010560""","""Elvis Grbac""","""Shane Matthews""","""Gunther Cunningham""","""Dick Jauron""","""Phil Luckett""","""CHI98""","""Soldier Field"""
"""1999_01_PIT_CLE""",1999,"""REG""",1,"""1999-09-12""","""Sunday""",null,"""PIT""",43,"""CLE""",0,"""Home""",-43,43,0,"""1999091213""",604,null,"""199909120cle""",null,"""190912005""",null,7,7,null,null,-6.0,null,null,37.0,null,null,1,"""outdoors""","""grass""",78,12,"""00-0015700""","""00-0004230""","""Kordell Stewart""","""Ty Detmer""","""Bill Cowher""","""Chris Palmer""","""Bob McElwee""","""CLE00""","""Cleveland Browns Stadium"""
"""1999_01_OAK_GB""",1999,"""REG""",1,"""1999-09-12""","""Sunday""",null,"""OAK""",24,"""GB""",28,"""Home""",4,52,0,"""1999091208""",602,null,"""199909120gnb""",null,"""190912009""",null,7,7,null,null,9.0,null,null,43.0,null,null,0,"""outdoors""","""grass""",67,10,"""00-0005741""","""00-0005106""","""Rich Gannon""","""Brett Favre""","""Jon Gruden""","""Ray Rhodes""","""Tony Corrente""","""GNB00""","""Lambeau Field"""
"""1999_01_BUF_IND""",1999,"""REG""",1,"""1999-09-12""","""Sunday""",null,"""BUF""",14,"""IND""",31,"""Home""",17,45,0,"""1999091202""",591,null,"""199909120clt""",null,"""190912011""",null,7,7,null,null,-3.0,null,null,45.5,null,null,1,"""dome""","""astroturf""",null,null,"""00-0005363""","""00-0010346""","""Doug Flutie""","""Peyton Manning""","""Wade Phillips""","""Jim Mora""","""Ron Blum""","""IND99""","""RCA Dome"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""2025_18_DAL_NYG""",2025,"""REG""",18,"""2026-01-04""","""Sunday""","""13:00""","""DAL""",null,"""NYG""",null,"""Home""",null,null,null,"""2025010411""",null,null,"""202501040nyg""",28684,"""401772963""",7001,6,6,null,null,null,null,null,null,null,null,1,"""outdoors""","""fieldturf""",null,null,null,null,null,null,"""Brian Schottenheimer""","""Brian Daboll""",null,"""NYC01""","""MetLife Stadium"""
"""2025_18_WAS_PHI""",2025,"""REG""",18,"""2026-01-04""","""Sunday""","""13:00""","""WAS""",null,"""PHI""",null,"""Home""",null,null,null,"""2025010412""",null,null,"""202501040phi""",28685,"""401772959""",7002,6,6,null,null,null,null,null,null,null,null,1,"""outdoors""","""grass""",null,null,null,null,null,null,"""Dan Quinn""","""Nick Sirianni""",null,"""PHI00""","""Lincoln Financial Field"""
"""2025_18_BAL_PIT""",2025,"""REG""",18,"""2026-01-04""","""Sunday""","""13:00""","""BAL""",null,"""PIT""",null,"""Home""",null,null,null,"""2025010413""",null,null,"""202501040pit""",28686,"""401772960""",7003,10,10,null,null,null,null,null,null,null,null,1,"""outdoors""","""grass""",null,null,null,null,null,null,"""John Harbaugh""",

In [5]:
# Select specific columns and convert to Spark DataFrame

nfl_data = nfl_data.select(
    "season",
    "week",
    "weekday",
    "home_team",
    "away_team",
    "gameday",
    "home_score",
    "away_score",
)

data_for_spark = nfl_data.to_dicts()

nfl_data = spark.createDataFrame(data_for_spark)
nfl_data.show()


+----------+---------+----------+----------+---------+------+----+-------+
|away_score|away_team|   gameday|home_score|home_team|season|week|weekday|
+----------+---------+----------+----------+---------+------+----+-------+
|        17|      MIN|1999-09-12|        14|      ATL|  1999|   1| Sunday|
|        17|       KC|1999-09-12|        20|      CHI|  1999|   1| Sunday|
|        43|      PIT|1999-09-12|         0|      CLE|  1999|   1| Sunday|
|        24|      OAK|1999-09-12|        28|       GB|  1999|   1| Sunday|
|        14|      BUF|1999-09-12|        31|      IND|  1999|   1| Sunday|
|         3|       SF|1999-09-12|        41|      JAX|  1999|   1| Sunday|
|        10|      CAR|1999-09-12|        19|       NO|  1999|   1| Sunday|
|        30|       NE|1999-09-12|        28|      NYJ|  1999|   1| Sunday|
|        25|      ARI|1999-09-12|        24|      PHI|  1999|   1| Sunday|
|        28|      DET|1999-09-12|        20|      SEA|  1999|   1| Sunday|
|        10|      BAL|199

In [6]:
# Change data types and names as needed

nfl_data = (
    (
        nfl_data.withColumn("season", nfl_data["season"].cast("double"))
        .withColumn("home_score", nfl_data["home_score"].cast("double"))
        .withColumn("away_score", nfl_data["away_score"].cast("double"))
    )
    .withColumnRenamed("home_score", "homePoints")
    .withColumnRenamed("away_score", "awayPoints")
    .withColumnRenamed("gameday", "game_date")
    .withColumnRenamed("home_team", "homeTeam")
    .withColumnRenamed("away_team", "awayTeam")
    .withColumnRenamed("weekday", "day_of_week_name")
)

In [7]:
nfl_data = nfl_data.withColumn(
    "Winner",
    when(col("homePoints") > col("awayPoints"), col("homeTeam"))
    .when(col("homePoints") < col("awayPoints"), col("awayTeam"))
    .otherwise("Tie"),
)

In [8]:
nfl_data.show()

+----------+--------+----------+----------+--------+------+----+----------------+------+
|awayPoints|awayTeam| game_date|homePoints|homeTeam|season|week|day_of_week_name|Winner|
+----------+--------+----------+----------+--------+------+----+----------------+------+
|      17.0|     MIN|1999-09-12|      14.0|     ATL|1999.0|   1|          Sunday|   MIN|
|      17.0|      KC|1999-09-12|      20.0|     CHI|1999.0|   1|          Sunday|   CHI|
|      43.0|     PIT|1999-09-12|       0.0|     CLE|1999.0|   1|          Sunday|   PIT|
|      24.0|     OAK|1999-09-12|      28.0|      GB|1999.0|   1|          Sunday|    GB|
|      14.0|     BUF|1999-09-12|      31.0|     IND|1999.0|   1|          Sunday|   IND|
|       3.0|      SF|1999-09-12|      41.0|     JAX|1999.0|   1|          Sunday|   JAX|
|      10.0|     CAR|1999-09-12|      19.0|      NO|1999.0|   1|          Sunday|    NO|
|      30.0|      NE|1999-09-12|      28.0|     NYJ|1999.0|   1|          Sunday|    NE|
|      25.0|     ARI|

In [9]:
# Filter data for GB and since 2021

nfl_data_filtered = nfl_data.filter(col("season") >= 2021).filter(
    when(col("homeTeam") == "GB", True)
    .when(col("awayTeam") == "GB", True)
    .otherwise(False)
)

In [10]:
print(nfl_data_filtered.count())
nfl_data_filtered.printSchema()

89
root
 |-- awayPoints: double (nullable = true)
 |-- awayTeam: string (nullable = true)
 |-- game_date: string (nullable = true)
 |-- homePoints: double (nullable = true)
 |-- homeTeam: string (nullable = true)
 |-- season: double (nullable = true)
 |-- week: long (nullable = true)
 |-- day_of_week_name: string (nullable = true)
 |-- Winner: string (nullable = true)



In [11]:
nfl_data_filtered = nfl_data_filtered.na.replace(float('nan'), None)

In [12]:
nfl_data_filtered.show(n=89, truncate=False)

+----------+--------+----------+----------+--------+------+----+----------------+------+
|awayPoints|awayTeam|game_date |homePoints|homeTeam|season|week|day_of_week_name|Winner|
+----------+--------+----------+----------+--------+------+----+----------------+------+
|3.0       |GB      |2021-09-12|38.0      |NO      |2021.0|1   |Sunday          |NO    |
|17.0      |DET     |2021-09-20|35.0      |GB      |2021.0|2   |Monday          |GB    |
|30.0      |GB      |2021-09-26|28.0      |SF      |2021.0|3   |Sunday          |GB    |
|17.0      |PIT     |2021-10-03|27.0      |GB      |2021.0|4   |Sunday          |GB    |
|25.0      |GB      |2021-10-10|22.0      |CIN     |2021.0|5   |Sunday          |GB    |
|24.0      |GB      |2021-10-17|14.0      |CHI     |2021.0|6   |Sunday          |GB    |
|10.0      |WAS     |2021-10-24|24.0      |GB      |2021.0|7   |Sunday          |GB    |
|24.0      |GB      |2021-10-28|21.0      |ARI     |2021.0|8   |Thursday        |GB    |
|7.0       |GB      |

In [13]:
write_single_csv(nfl_data_filtered, "win_tracker/nfl/nfl_data.csv")


CSV written to: win_tracker/nfl/nfl_data.csv
